## LLM Router Chains

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router import MultiPromptChain
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
llm = ChatOpenAI()

## Establish the templates for the two routes

In [21]:
beginner_template = "You are a linguistics 101 instructor. You want to explain complex liguistics concepts in the simplest ways possible. You assume no prior knowledge of linguistics. Here is your question: \n{input}"
expert_template = "You are a linguistics professor who speaks at academic conferences. You can assume anyone you answer has a PhD in linguistics. Here is your question: \n{input}"

## Establish route prompt information

In [22]:
prompt_infos = [
    {"name":"beginner linguistics", 
    "description": "answers basic linguistics question",
    "template":beginner_template,},
        {"name":"expert template", 
    "description": "answers expert linguistics questions",
    "template": expert_template,},
]
# the data type is list

In [23]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm =llm, prompt=prompt)
    destination_chains[name] = chain

In [24]:
type(destination_chains)
# type should be dict (curly braces)


dict

## Establish default prompt

In [25]:
default_prompt = ChatPromptTemplate.from_template("{input}")
# {input} refers back to the two route templates
default_chain = LLMChain(llm=llm, prompt=default_prompt)

## Set up routing

In [26]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)
# this is an elaborate prompt that instructs the LLM how to do the routing. It needs to be modified.

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [27]:
destinations = [f"{p['name']}: {p['description']}"  for p in prompt_infos]

In [28]:
destination_str = "\n".join(destinations)
# This is how it has to be formatted to fit into the multi prompt router template: not as two strings, but as one string with a line break in between

In [29]:
print(destination_str)

beginner linguistics: answers basic linguistics question
expert template: answers expert linguistics questions


## Creating router template

In [30]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations = destination_str
)

In [31]:
print(router_template)
# the router template now has the appropriate routing information

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
beginner linguistics: answers basic linguistics question
expert template: answers expert linguistics 

## Creating router prompt

In [32]:
router_prompt = PromptTemplate(template= router_template,
                               input_variables = ['input'],
                               output_parser = RouterOutputParser())

## Creating router chain

In [33]:
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [34]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains = destination_chains,
                         default_chain=default_chain,
                         verbose = True)

## Running the chain

In [35]:
chain.run("what's a phoneme?")

/home/codespace/.python/current/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/codespace/.python/current/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(




> Entering new MultiPromptChain chain...
beginner linguistics: {'input': "what's a phoneme?"}
> Finished chain.


'A phoneme is like a building block of sound in a language. It\'s the smallest unit of sound that can change the meaning of a word. For example, if we take the word "cat" and change the first sound to "b" instead of "c," we get a different word, "bat." So, the "c" and "b" sounds are different phonemes because they can create different words when used in the same position.'

In [41]:
chain.run("Critique Noam Chomsky's 'Syntacic Structures'")
# it's not activating the expert chain for some reason



> Entering new MultiPromptChain chain...


/home/codespace/.python/current/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


None: {'input': "Critique Noam Chomsky's 'Syntacic Structures'"}
> Finished chain.


'Noam Chomsky\'s "Syntactic Structures" is a groundbreaking work in the field of linguistics, introducing many new concepts and ideas that have had a lasting impact on the study of language. However, it is not without its critiques.\n\nOne of the main criticisms of "Syntactic Structures" is its reliance on transformational grammar, which some linguists argue is too abstract and divorced from actual language use. Chomsky\'s focus on the underlying structure of sentences, rather than their surface forms, has been seen by some as overly theoretical and disconnected from the realities of language production and comprehension.\n\nAnother critique of "Syntactic Structures" is its lack of empirical evidence. Chomsky\'s arguments and claims are mainly based on intuitive judgments and examples, rather than extensive data analysis. This has led some linguists to question the validity and generalizability of his theories, arguing that they are not sufficiently grounded in real-world language phen